In [17]:
!pip uninstall open3d

^C


In [18]:
!conda install -c open3d-admin open3d


^C


In [4]:
import os
import matplotlib.pyplot as plt
import pandas as pd
import torch
import torch.nn as nn
import numpy as np
from glob import glob
# import open3d as o3d
# print("Open3D Version: " + o3d.__version__)
# o3d.visualization.webrtc_server.enable_webrtc()
kitti_velodyne_path= r"D:\introspectionBase\datasets\Kitti\raw\training\velodyne"


In [15]:
class SpatialAttention(nn.Module):
    def __init__(self, kernel_size=7):
        super(SpatialAttention, self).__init__()
        self.conv1 = nn.Conv2d(2, 1, kernel_size, padding=kernel_size // 2, bias=False)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        avg_out = torch.mean(x, dim=1, keepdim=True)
        print(avg_out.shape)
        max_out, _ = torch.max(x, dim=1, keepdim=True)
        print(max_out.shape)
        x = torch.cat([avg_out, max_out], dim=1)
        print(x.shape)
        x = self.conv1(x)
        return self.sigmoid(x)


import torch
import torch.nn as nn
import torch.nn.functional as F

class ChannelAttention(nn.Module):
    def __init__(self, in_channels, reduction=16):
        super(ChannelAttention, self).__init__()
        self.avg_pool = nn.AdaptiveAvgPool2d(1)
        self.max_pool = nn.AdaptiveMaxPool2d(1)

        self.fc1 = nn.Conv2d(in_channels, in_channels // reduction, 1, bias=False)
        self.relu = nn.ReLU()
        self.fc2 = nn.Conv2d(in_channels // reduction, in_channels, 1, bias=False)

        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        avg_out = self.fc2(self.relu(self.fc1(self.avg_pool(x))))
        max_out = self.fc2(self.relu(self.fc1(self.max_pool(x))))
        out = avg_out + max_out
        return self.sigmoid(out)

In [22]:
class NonLocalBlock(nn.Module):
    def __init__(self, in_channels, inter_channels=None,apply=True):
        super(NonLocalBlock, self).__init__()
        self.in_channels = in_channels
        self.inter_channels = inter_channels if inter_channels is not None else in_channels
        self.apply = apply

        self.g = nn.Conv2d(in_channels=self.in_channels, out_channels=self.inter_channels, kernel_size=1, stride=1, padding=0)
        self.theta = nn.Conv2d(in_channels=self.in_channels, out_channels=self.inter_channels, kernel_size=1, stride=1, padding=0)
        self.phi = nn.Conv2d(in_channels=self.inter_channels, out_channels=self.inter_channels, kernel_size=1, stride=1, padding=0)

        self.W = nn.Conv2d(in_channels=self.inter_channels, out_channels=self.in_channels, kernel_size=1, stride=1, padding=0)
        nn.init.constant_(self.W.weight, 0)
        nn.init.constant_(self.W.bias, 0)

    def forward(self, x):
        batch_size = x.size(0)
        g_x = self.g(x).view(batch_size, self.inter_channels, -1)
        g_x = g_x.permute(0, 2, 1)

        theta_x = self.theta(x).view(batch_size, self.inter_channels, -1)
        theta_x = theta_x.permute(0, 2, 1)
        phi_x = self.phi(x).view(batch_size, self.inter_channels, -1)

        f = torch.matmul(theta_x, phi_x)
        f_div_C = F.softmax(f, dim=-1)

        y = torch.matmul(f_div_C, g_x)
        y = y.permute(0, 2, 1).contiguous()
        y = y.view(batch_size, self.inter_channels, *x.size()[2:])
        W_y = self.W(y)
        z = W_y + x

        return z if self.apply else W_y


In [23]:
attention = torch.ones(1, 256, 1, 1)

# Input feature map of shape [1, 256, 62, 54]
x = torch.ones(1, 256, 62, 54)

# This operation is valid due to broadcasting
result = NonLocalBlock(256)(x)
result.shape

torch.Size([1, 256, 62, 54])

In [33]:
from torchvision import models
t = models.resnet18()


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [2]:
def get_rotation_matrix_from_corners(corners):
    """
    Get the rotation matrix from 8 corners of a 3D bounding box.
    
    Parameters:
        corners (numpy.ndarray): 3x8 array containing the coordinates of the 8 corners.
        
    Returns:
        numpy.ndarray: 3x3 rotation matrix.
    """
    
    # Assume the corners are ordered such that:
    # - corners[:, 0] and corners[:, 1] are opposite corners on the "bottom" face of the box
    # - corners[:, 0] and corners[:, 4] are opposite corners on the "front" face of the box
    # - corners[:, 0] and corners[:, 3] are opposite corners on the "left" face of the box
    
    # Compute the vectors representing the edges
    edge1 = corners[:, 1] - corners[:, 0]  # Vector from corner 0 to corner 1
    edge2 = corners[:, 4] - corners[:, 0]  # Vector from corner 0 to corner 4
    edge3 = corners[:, 3] - corners[:, 0]  # Vector from corner 0 to corner 3
    
    # Normalize the vectors
    edge1 /= np.linalg.norm(edge1)
    edge2 /= np.linalg.norm(edge2)
    edge3 /= np.linalg.norm(edge3)
    
    # Construct the rotation matrix
    R = np.column_stack((edge1, edge2, edge3))
    
    return R
def is_inside_ellipse(x, y, a, b):
    return (x**2 / a**2) + (y**2 / b**2) <= 1
def is_outside_ellipse(x, y, a, b):
    return (x**2 / a**2) + (y**2 / b**2) > 1
def set_custom_view(vis):
    ctr = vis.get_view_control()
    
    # Create an extrinsic matrix for camera placement
    extrinsic = np.eye(4)
    extrinsic[0:3, 3] = [-10, 0, 0]  # Set camera position (x, y, z)
    
    # Create a rotation matrix for 30-degree downward view
    rotation = np.array([
        [1, 0, 0],
        [0, np.cos(np.radians(-160)), -np.sin(np.radians(-160))],
        [0, np.sin(np.radians(-160)), np.cos(np.radians(-160))]
    ])
    
    # Apply rotation to the extrinsic matrix
    extrinsic[0:3, 0:3] = rotation
    
    # Set the extrinsic matrix to the camera parameters
    cam_params = ctr.convert_to_pinhole_camera_parameters()
    cam_params.extrinsic = extrinsic
    ctr.convert_from_pinhole_camera_parameters(cam_params)
    opt = vis.get_render_option()
    opt.background_color = np.asarray([0.5, 0.5, 0.5])

def load_velodyne_points(filename):
    points = np.fromfile(filename, dtype=np.float32).reshape(-1, 4)
    return points
def read_calib_file(calib_file):
    with open(calib_file, 'r') as f:
        lines = f.readlines()
        calibration = {}

        for line in lines:
            # print(line)
            try:
                key, value = line.split(':')
            except ValueError:
                continue
            calibration[key] = np.array([float(x) for x in value.strip().split()])
        return calibration
def compute_colors_from_distance(points,max_distance):
    #If no disatance given return all black
    if max_distance==None:
        return np.zeros((points.shape[0],3))
    distances = np.linalg.norm(points[:, :3], axis=1)
    normalized_distances = distances / max_distance
    return plt.cm.jet(normalized_distances)[:,:3]
def create_point_cloud(points,distance=None):
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(points[:, :3])
    colors = compute_colors_from_distance(points,distance)
    pcd.colors = o3d.utility.Vector3dVector(colors)
    return pcd
def plot_bounding_box_from_corners(corners,offset =0, calib=None, color=[0, 1, 0]):
    # Define the lines connecting the corners based on their indices

    lines = [[0, 1], [1, 2], [2, 3], [3, 0],
             [4, 5], [5, 6], [6, 7], [7, 4],
             [0, 4], [1, 5], [2, 6], [3, 7]]

    # Create a LineSet object
    line_set = o3d.geometry.LineSet()
    line_set.points = o3d.utility.Vector3dVector(corners)
    line_set.lines = o3d.utility.Vector2iVector(lines)
    line_set.colors = o3d.utility.Vector3dVector([color for i in range(len(lines))])

    return line_set
def filter_objects_outside_ellipse(objects, a, b,offset=5,axis=0):
    """
    Filters ground truth objects to only include those outside a specified ellipse.

    Parameters:
        objects (list): The input objects, each as a dictionary.
        a (float): Semi-major axis length of the ellipse.
        b (float): Semi-minor axis length of the ellipse.

    Returns:
        list: The filtered objects.
    """
    filtered_objects = []
    
    for obj in objects:
        # print(obj)
        if 'corners' not in obj.keys():
            x, y = obj['x'], obj['y']
            
            # Generate corner points for the box
            dx, dy = obj['dx'], obj['dy']

        else:
            x,y,_ = obj['real_center']
        dx, dy = obj['dx'], obj['dy']
    
        # Check if any corner point is inside the ellipse
        corners = np.array([
                [x - dx/2, y - dy/2],
                [x - dx/2, y + dy/2],
                [x + dx/2, y - dy/2],
                [x + dx/2, y + dy/2]
            ])
        adjusted_corners_x = corners[:, axis] + offset
        inside_ellipse = is_inside_ellipse(adjusted_corners_x, corners[:, 1], a, b)
        if np.any(inside_ellipse):
            filtered_objects.append(obj)
    # print(filtered_objects)
    return filtered_objects

def create_oriented_bounding_box_gt(box_params, color=(0, 1, 0),calib=None, offset=0,axis=0):
    center = np.array([box_params['x'], box_params['y'], box_params['z']/2])
    extent = np.array([box_params['dx'], box_params['dy'], box_params['dz']])
    R = box_params['yaw']

    # Tr_velo_to_cam = calib['Tr_velo_to_cam'].reshape(3, 4)
    if("corners" in box_params.keys()):
    # # Convert center to homogeneous coordinates and transform to LiDAR coordinates
    # point_homogeneous = np.hstack((center, np.ones(1)))
    # center_tr = transform_camera_to_lidar(point_homogeneous, Tr_velo_to_cam)
        center = box_params['real_center']
        coners = box_params['corners']
        # coners[:,2] /=2
        coners[:,axis] += offset
        return plot_bounding_box_from_corners(coners,offset=offset, calib=calib, color=color)
        # R_empty = np.eye(3)
        # obb = o3d.geometry.OrientedBoundingBox(center=real_center, R=R, extent=extent)
        # cntr = o3d.geometry.OrientedBoundingBox(center=real_center, R=R, extent=[0.1,0.1,0.1])
        # cntr.color = (1,1,1)
        # obb.color = color
        # return obb, cntr
    # Rotate around the Y-axis for KITTI's rotation_y
    # print("Plotting from center")
    # Create OrientedBoundingBox
    obb = o3d.geometry.OrientedBoundingBox(center=center, R=R, extent=extent)
    cntr = o3d.geometry.OrientedBoundingBox(center=center, R=R, extent=[0.1,0.1,0.1])
    cntr.color = (1,1,1)
    obb.color = color 
    
    return obb
def filter_points_inside_ellipse(points, a, b,offset=5):
    x, y, z = points[:, 0]+offset, points[:, 1], points[:, 2]
    inside = is_inside_ellipse(x, y, a, b)
    return points[inside]
def filter_points_outside_ellipse(points, a, b,offset=5):
    x, y, z = points[:, 0]+offset, points[:, 1], points[:, 2]
    outside = is_outside_ellipse(x, y, a, b)
    return points[outside]
def read_kitti_label_file(bin_file_path, filter_classes=['Car', 'Pedestrian', 'Cyclist']):
    """
    Reads a KITTI label .txt file and returns a list of dictionaries containing
    object type and bounding box parameters, filtered by specified classes.

    Parameters:
        txt_file_path (str): Path to the KITTI label .txt file.
        filter_classes (List[str]): List of classes to include.

    Returns:
        List[Dict]: A list of dictionaries containing 'type', 'x', 'y', 'z', 'dx', 'dy', 'dz', and 'yaw'.
    """
    txt_file_path = bin_file_path.replace('.bin', '.txt').replace("velodyne","label_2")
    calib_file_path = bin_file_path.replace('.bin', '.txt').replace("velodyne","calib")
    calib = read_calib_file(calib_file_path)
    objects = []
    from math import cos,sin

    with open(txt_file_path, 'r') as f:
        lines = f.readlines()
        for line in lines:
            tokens = line.strip().split(' ')
            obj_type = tokens[0]
            if obj_type not in filter_classes:
                continue
            # print(line)
            dimensions_height, dimensions_width, dimensions_length = map(float, tokens[8:11])
            location_x, location_y, location_z = map(float, tokens[11:14])
            center_tr = np.array([location_x, location_y, location_z])
            rotation_y = float(tokens[14])
            l_div_2 = dimensions_length / 2
            x_corners = [l_div_2, l_div_2, -l_div_2, -l_div_2, l_div_2, l_div_2, -l_div_2, -l_div_2]
            w_div_2 = dimensions_width / 2
            y_corners = [0, 0, 0, 0, -dimensions_height, -dimensions_height, -dimensions_height, -dimensions_height]
            z_corners = [w_div_2, -w_div_2, -w_div_2, w_div_2, w_div_2, -w_div_2, -w_div_2, w_div_2]
            corner_matrix = np.array([x_corners, y_corners, z_corners])
            R = np.array([[cos(rotation_y),0,sin(rotation_y)],[0,1,0],[-sin(rotation_y),0,cos(rotation_y)]])
            rotated_corners = np.matmul(R,corner_matrix)
            translated_corners = rotated_corners + center_tr.reshape(3,1)
            
            Tr_velo_to_cam = calib_data['Tr_velo_to_cam'].reshape(3, 4)
            Tr_velo_to_cam_extended = np.eye(4)  # Create a 4x4 identity matrix
            Tr_velo_to_cam_extended[:3, :] = Tr_velo_to_cam  # Replace the top-left 3x4 block
            
            # rotation_component = np.eye(4)
            # rotation_component[:3,:3] = Tr_velo_to_cam_extended[:3,:3]
            
            # translation_component = np.eye(4)
            # translation_component[:3,3] = Tr_velo_to_cam_extended[:3,3]
            
            # inverse_rotation = np.linalg.inv(rotation_component)
            # inverse_translation =  -translation_component
            
            # new_transform = np.eye(4)
            # new_transform[:3,:3] = inverse_rotation[:3,:3]
            # new_transform[:3,3] = inverse_translation[:3,3]

            # print(new_transform.shape,translated_corners.shape)
            T_inv = np.linalg.inv(Tr_velo_to_cam_extended)
            Homogeneous_corners = np.ones((4,8))
            Homogeneous_corners[:3,:] = translated_corners
            translated_corners = np.matmul(T_inv,Homogeneous_corners)[:3,:]
            real_center = np.mean(translated_corners,axis=1)
            # print(translated_corners.shape)
            # real_center = np.mean(translated_corners,axis=1)
            # print(real_center)
            objects.append({
                'type': obj_type,
                'x': real_center[0],
                'y': real_center[1],
                'z': real_center[2],
                'dx': dimensions_length,
                'dy': dimensions_height,
                'dz': dimensions_width,
                'yaw': o3d.geometry.get_rotation_matrix_from_xyz((0,rotation_y,0)),
                'corners': translated_corners.T,
                'real_center': real_center
            })
    return objects
def create_oriented_bounding_box(box_params,offset=100,axis=0,calib=None,color=(1, 0, 0)):
    offset_array = np.zeros(3)
    offset_array[axis] = offset
    center = np.array([box_params[0], box_params[1], box_params[2]/2])
    extent = np.array([box_params[3], box_params[4], box_params[5]])
    R = o3d.geometry.get_rotation_matrix_from_xyz((0,0,box_params[6]))

    # Transform_lidar_to_cam = calib['Tr_velo_to_cam'].reshape(3, 4)
    # extended_transform = np.eye(4)
    # extended_transform[:3, :] = Transform_lidar_to_cam
    # corners = np.array([center + np.dot(R, np.array([x, y, z])) for x in [-extent[0], extent[0]] for y in [-extent[1], extent[1]] for z in [-extent[2], extent[2]]])
    # center_from_corners = np.mean(corners, axis=0)
    # center  = center_from_corners
    center += offset_array
    R_empty = np.eye(3)
    obb = o3d.geometry.OrientedBoundingBox(center=center, R=R, extent=extent)
    obb.color = (1, 0, 0)  # Red color
    return obb
def create_oriented_bounding_box_nuscenes(detection, offset=100, axis=0, color=(1, 0, 0),calib=None):
    offset_array = np.zeros(3)
    offset_array[axis] = offset

    # Extract parameters from the detection array
    x, y, z, dx, dy, dz, r, _, _ = detection

    # Define the center and extent of the box
    center = np.array([x, y, z/2])
    extent = np.array([dx, dy, dz])

    # Create the rotation matrix from the heading angle
    R = o3d.geometry.get_rotation_matrix_from_xyz((0, 0, 0 ))

    # Apply the offset to the center
    center += offset_array

    # Create the Oriented Bounding Box
    obb = o3d.geometry.OrientedBoundingBox(center=center, R=R, extent=extent)
    obb.color = color  # Set the color
    obb_for_center = o3d.geometry.OrientedBoundingBox(center=center, R=R, extent=[0.1,0.1,0.1])
    obb_for_center.color = (1,1,1)
    return obb , obb_for_center
def rotate_points(points, R):
    return np.dot(points, R.T)

def match_detections_3d(ground_truths, predictions, iou_threshold=0.5):
    matches = []
    unmatched_ground_truths = []
    unmatched_predictions = list(predictions)

    for gt in ground_truths:
        max_iou = -1
        max_iou_idx = -1
        for idx, pred in enumerate(unmatched_predictions):
            current_iou = calculate_iou_3d(gt, pred)
            if current_iou > max_iou:
                max_iou = current_iou
                max_iou_idx = idx
        # print("For gt",gt,"max iou is",max_iou)
        if max_iou >= iou_threshold:
            matches.append((gt, unmatched_predictions[max_iou_idx]))
            del unmatched_predictions[max_iou_idx]
        else:
            unmatched_ground_truths.append(gt)

    return matches, unmatched_ground_truths, unmatched_predictions
def calculate_iou_3d(box1,box2):
    # Create corner points for both boxes
    if type(box1) == o3d.geometry.OrientedBoundingBox:
        center1 = np.array(box1.center)
        dimensions1 = np.array(box1.extent)
        half_dims1 = dimensions1 / 2
        corners1 = np.array([np.array([x, y, z]) for x in [-half_dims1[0], half_dims1[0]] for y in [-half_dims1[1], half_dims1[1]] for z in [-half_dims1[2], half_dims1[2]]])
        corners1 = rotate_points(corners1, box1.R) + center1

    else:
        #get corners from lineset
        # print("Getting corners from lineset")
        corners1 = np.array(box1.points)
        dimensions_from_corners = np.max(corners1,axis=0) - np.min(corners1,axis=0)
        dimensions1 = dimensions_from_corners
        
    if type(box2)== o3d.geometry.OrientedBoundingBox:
        center2 = np.array(box2.center)
        dimensions2 = np.array(box2.extent)
        half_dims2 = dimensions2 / 2
        corners2 = np.array([np.array([x, y, z]) for x in [-half_dims2[0], half_dims2[0]] for y in [-half_dims2[1], half_dims2[1]] for z in [-half_dims2[2], half_dims2[2]]])
        # Rotate and translate corners
        corners2 = rotate_points(corners2, box2.R) + center2
    else:
        #get corners from lineset
        # print("Getting corners from lineset")

        corners2 = np.array(box2.points)
        dimensions_from_corners = np.max(corners2,axis=0) - np.min(corners2,axis=0)
        dimensions2 = dimensions_from_corners
    # Calculate axis-aligned bounding boxes for intersection
    min_bound1 = np.min(corners1, axis=0)
    max_bound1 = np.max(corners1, axis=0)
    min_bound2 = np.min(corners2, axis=0)
    max_bound2 = np.max(corners2, axis=0)
    
    # Calculate intersection
    min_intersect = np.maximum(min_bound1, min_bound2)
    max_intersect = np.minimum(max_bound1, max_bound2)
    intersection_dims = np.maximum(0, max_intersect - min_intersect)
    intersection_volume = np.prod(intersection_dims)
    
    # Calculate volumes of the individual boxes
    vol1 = np.prod(dimensions1)
    vol2 = np.prod(dimensions2)
    
    # Calculate IoU
    iou = intersection_volume / (vol1 + vol2 - intersection_volume)
    return iou


def is_point_inside_obb(obb, point):
    """
    Check if a point is inside an oriented bounding box (OBB).
    
    Parameters:
    - obb: Open3D OrientedBoundingBox object.
    - point: NumPy array of shape (3,) representing the point.
    
    Returns:
    - bool: True if the point is inside the OBB, False otherwise.
    """
    # Transform the point to the OBB's local coordinate system
    point_local = np.linalg.inv(obb.R).dot((point[:3] - obb.center))
    
    # Check if the transformed point is inside the axis-aligned box
    return np.all(np.abs(point_local) <= (obb.extent / 2))

def remove_points_inside_obbs(point_cloud, obbs):
    """
    Remove points inside oriented bounding boxes from a point cloud.
    
    Parameters:
    - point_cloud: NumPy array of shape (N, 3), where N is the number of points.
    - obbs: List of Open3D OrientedBoundingBox objects.
    
    Returns:
    - filtered_point_cloud: NumPy array containing points outside the bounding boxes.
    """
    mask = np.ones(point_cloud.shape[0], dtype=bool)
    
    for obb in obbs:
        for i, point in enumerate(point_cloud):
            if is_point_inside_obb(obb, point):
                mask[i] = False
    
    filtered_point_cloud = point_cloud[mask]
    
    return filtered_point_cloud



In [3]:
import os
import matplotlib.pyplot as plt
import pandas as pd
import torch
import torch.nn as nn
import numpy as np
from glob import glob
import open3d as o3d
import mmdet3d
import numpy as np
import pandas as pd
import cv2
from plyfile import PlyData
from mmcv.transforms.base import BaseTransform
from mmengine.registry import TRANSFORMS, VISUALIZERS
from mmengine.structures import InstanceData
from mmdet3d.utils import register_all_modules
from mmdet3d.apis import inference_detector, init_model
from mmdet3d.evaluation.metrics.nuscenes_metric import NuScenesMetric
from mmdet3d.datasets import NuScenesDataset
from mmdet3d.structures import Det3DDataSample, LiDARInstance3DBoxes
from mmdet3d.visualization import Det3DLocalVisualizer
from mmdet3d.structures import LiDARInstance3DBoxes
def load_velodyne_points(filename):
    points = np.fromfile(filename, dtype=np.float32).reshape(-1, 4)
    return points
def read_calib_file(calib_file):
    with open(calib_file, 'r') as f:
        lines = f.readlines()
        calibration = {}

        for line in lines:
            # print(line)
            try:
                key, value = line.split(':')
            except ValueError:
                continue
            calibration[key] = np.array([float(x) for x in value.strip().split()])
        return calibration
def read_kitti_label_file(bin_file_path, filter_classes=['Car', 'Pedestrian', 'Cyclist']):
    """
    Reads a KITTI label .txt file and returns a list of dictionaries containing
    object type and bounding box parameters, filtered by specified classes.

    Parameters:
        txt_file_path (str): Path to the KITTI label .txt file.
        filter_classes (List[str]): List of classes to include.

    Returns:
        List[Dict]: A list of dictionaries containing 'type', 'x', 'y', 'z', 'dx', 'dy', 'dz', and 'yaw'.
    """
    txt_file_path = bin_file_path.replace('.bin', '.txt').replace("velodyne","label_2")
    calib_file_path = bin_file_path.replace('.bin', '.txt').replace("velodyne","calib")
    calib = read_calib_file(calib_file_path)
    objects = []
    from math import cos,sin

    with open(txt_file_path, 'r') as f:
        lines = f.readlines()
        for line in lines:
            tokens = line.strip().split(' ')
            obj_type = tokens[0]
            if obj_type not in filter_classes:
                continue
            # print(line)
            dimensions_height, dimensions_width, dimensions_length = map(float, tokens[8:11])
            location_x, location_y, location_z = map(float, tokens[11:14])
            center_tr = np.array([location_x, location_y, location_z])
            rotation_y = float(tokens[14])
            l_div_2 = dimensions_length / 2
            x_corners = [l_div_2, l_div_2, -l_div_2, -l_div_2, l_div_2, l_div_2, -l_div_2, -l_div_2]
            w_div_2 = dimensions_width / 2
            y_corners = [0, 0, 0, 0, -dimensions_height, -dimensions_height, -dimensions_height, -dimensions_height]
            z_corners = [w_div_2, -w_div_2, -w_div_2, w_div_2, w_div_2, -w_div_2, -w_div_2, w_div_2]
            corner_matrix = np.array([x_corners, y_corners, z_corners])
            R = np.array([[cos(rotation_y),0,sin(rotation_y)],[0,1,0],[-sin(rotation_y),0,cos(rotation_y)]])
            rotated_corners = np.matmul(R,corner_matrix)
            translated_corners = rotated_corners + center_tr.reshape(3,1)
            
            Tr_velo_to_cam = calib['Tr_velo_to_cam'].reshape(3, 4)
            Tr_velo_to_cam_extended = np.eye(4)  # Create a 4x4 identity matrix
            Tr_velo_to_cam_extended[:3, :] = Tr_velo_to_cam  # Replace the top-left 3x4 block
            
            # rotation_component = np.eye(4)
            # rotation_component[:3,:3] = Tr_velo_to_cam_extended[:3,:3]
            
            # translation_component = np.eye(4)
            # translation_component[:3,3] = Tr_velo_to_cam_extended[:3,3]
            
            # inverse_rotation = np.linalg.inv(rotation_component)
            # inverse_translation =  -translation_component
            
            # new_transform = np.eye(4)
            # new_transform[:3,:3] = inverse_rotation[:3,:3]
            # new_transform[:3,3] = inverse_translation[:3,3]

            # print(new_transform.shape,translated_corners.shape)
            T_inv = np.linalg.inv(Tr_velo_to_cam_extended)
            Homogeneous_corners = np.ones((4,8))
            Homogeneous_corners[:3,:] = translated_corners
            translated_corners = np.matmul(T_inv,Homogeneous_corners)[:3,:]
            real_center = np.mean(translated_corners,axis=1)
            # print(translated_corners.shape)
            # real_center = np.mean(translated_corners,axis=1)
            # print(real_center)
            objects.append({
                'type': obj_type,
                'x': real_center[0],
                'y': real_center[1],
                'z': real_center[2],
                'dx': dimensions_length,
                'dy': dimensions_height,
                'dz': dimensions_width,
                'yaw': o3d.geometry.get_rotation_matrix_from_xyz((0,rotation_y,0)),
                'corners': translated_corners.T,
                'real_center': real_center
            })
    return objects

d:\mmdetection3d\mmdet3d\evaluation\functional\kitti_utils\eval.py:10: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  def get_thresholds(scores: np.ndarray, num_gt, num_sample_pts=41):


In [4]:
import random
import os
import numpy as np
from tqdm.auto import tqdm

kitti_velodyne_path= r"D:\introspectionBase\datasets\Kitti\raw\training\velodyne"
dataset_path = r"D:\Introspect3D\custom_dataset\pointpillars_kitti_class3"
flag = 0

def myhook(module, input, output):
    global im_name,dataset_path,flag
    # print(output[0].shape,output[1].shape,output[2].shape)
    numpy_output = output[2].squeeze().cpu().detach().numpy()
    # print(numpy_output.shape)
    # print(f"Saving to {os.path.join(dataset_path,im_name.replace('.png','.npy'))}")
    if(flag):
        np.save(os.path.join(dataset_path,'features',im_name.replace(".png",".npy")),numpy_output)
    else:
        np.save(os.path.join(dataset_path,'features_removed',im_name.replace(".png",".npy")),numpy_output)
def read_calib_file(calib_file):
    with open(calib_file, 'r') as f:
        lines = f.readlines()
        calibration = {}

        for line in lines:
            # print(line)
            try:
                key, value = line.split(':')
            except ValueError:
                continue
            calibration[key] = np.array([float(x) for x in value.strip().split()])
        return calibration
if __name__ == "__main__":
    # build the model from a config file and a checkpoint file
    config_file = r'D:\mmdetection3d\configs\pointpillars/pointpillars_hv_secfpn_8xb6-160e_kitti-3d-3class.py'
    # download the checkpoint from model zoo and put it in `checkpoints/`
    checkpoint = r'D:\mmdetection3d/ckpts/hv_pointpillars_secfpn_6x8_160e_kitti-3d-3class_20220301_150306-37dc2420.pth'
    model = init_model(config_file, checkpoint, device='cuda:0')

    hook = model.backbone.register_forward_hook(myhook)
    score_threshold = 0.5
    with tqdm(total=7480) as pbar:
        for i in range(7480):
            test_num  = i
            filename = os.path.join(kitti_velodyne_path, f'{test_num:06}.bin')
            calib_data = read_calib_file(filename.replace('.bin', '.txt').replace("velodyne","calib"))
            points = load_velodyne_points(filename)

            im_name = f'{test_num:06}.png'           
            # Transformation matrix is in (3x4) shape, so we extend it to (4x4) by adding a row of [0,0,0,1] for homogeneous coordinates
            Tr_velo_to_cam = calib_data['Tr_velo_to_cam'].reshape(3, 4)
            Tr_velo_to_cam_extended = np.eye(4)  # Create a 4x4 identity matrix
            Tr_velo_to_cam_extended[:3, :] = Tr_velo_to_cam  # Replace the top-left 3x4 block
            
            max_distance = np.max(np.linalg.norm(points[:, :3], axis=1))
            a, b = 25, 15  # Semi-major and semi-minor axes
            filtered_points = filter_points_inside_ellipse(points, a, b,offset=-10) #filter_points_inside_pyramid(points, min_x, max_x, min_y,max_y ) #
            filtered_pcd = create_point_cloud(filtered_points,distance=max_distance)
            outside_points = filter_points_outside_ellipse(filtered_points, a, b,offset=-10)
            outside_pcd = create_point_cloud(outside_points)
            flag = 1
            res_f,data_f = inference_detector(model, filtered_points)
            pred_boxes_box_f = res_f.pred_instances_3d.bboxes_3d.tensor.cpu().numpy()
            pred_boxes_score_f = res_f.pred_instances_3d.scores_3d.cpu().numpy()
            filtered_indices = np.where(pred_boxes_score_f >= score_threshold)[0]
            filtered_boxes = pred_boxes_box_f[filtered_indices]
            obb_list_f = [create_oriented_bounding_box(box,offset=0,calib=calib_data) for box in filtered_boxes]
            removed_object_points = remove_points_inside_obbs(filtered_points, obb_list_f)
            flag = 0
            # removed_pcd = create_point_cloud(removed_object_points,distance=max_distance)
            one_more_chance_res, one_more_chance_data = inference_detector(model, removed_object_points)
            # one_more_chance_pred_boxes_box = one_more_chance_res.pred_instances_3d.bboxes_3d.tensor.cpu().numpy()
            # one_more_chance_pred_boxes_score = one_more_chance_res.pred_instances_3d.scores_3d.cpu().numpy()
            # filtered_indices = np.where(one_more_chance_pred_boxes_score >= score_threshold)[0]
            # one_more_chance_filtered_boxes = one_more_chance_pred_boxes_box[filtered_indices]
            # one_more_chance_obb_list_f = [create_oriented_bounding_box(box,offset=0,calib=calib_data,color=(1,0.647,0)) for box in one_more_chance_filtered_boxes]

            # test a single sample
            # print(points.shape)
            # points = np.expand_dims(points,axis=0)
            # res = inference_detector(model,points)  
            pbar.update(1)
    hook.remove()

d:\mmdetection3d\mmdet3d\models\dense_heads\anchor3d_head.py:92: UserWarning: dir_offset and dir_limit_offset will be depressed and be incorporated into box coder in the future
  warnings.warn(


Loads checkpoint by local backend from path: D:\mmdetection3d/ckpts/hv_pointpillars_secfpn_6x8_160e_kitti-3d-3class_20220301_150306-37dc2420.pth


  0%|          | 0/7480 [00:00<?, ?it/s]

c:\Users\yatba\anaconda3\envs\openmmlab\lib\site-packages\torch\functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at C:\cb\pytorch_1000000000000\work\aten\src\ATen\native\TensorShape.cpp:3484.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


KeyboardInterrupt: 

In [4]:
    config_file = r'D:\mmdetection3d\configs\pointpillars/pointpillars_hv_secfpn_8xb6-160e_kitti-3d-3class.py'
    # download the checkpoint from model zoo and put it in `checkpoints/`
    checkpoint = r'D:\mmdetection3d/ckpts/hv_pointpillars_secfpn_6x8_160e_kitti-3d-3class_20220301_150306-37dc2420.pth'
    model = init_model(config_file, checkpoint, device='cuda:0')


Loads checkpoint by local backend from path: D:\mmdetection3d/ckpts/hv_pointpillars_secfpn_6x8_160e_kitti-3d-3class_20220301_150306-37dc2420.pth


d:\mmdetection3d\mmdet3d\models\dense_heads\anchor3d_head.py:92: UserWarning: dir_offset and dir_limit_offset will be depressed and be incorporated into box coder in the future
  warnings.warn(


In [6]:
model

VoxelNet(
  (data_preprocessor): Det3DDataPreprocessor(
    (voxel_layer): VoxelizationByGridShape(voxel_size=[0.16, 0.16, 4], grid_shape=[432, 496, 1], point_cloud_range=[0, -39.68, -3, 69.12, 39.68, 1], max_num_points=32, max_voxels=(16000, 40000), deterministic=True)
  )
  (backbone): SECOND(
    (blocks): ModuleList(
      (0): Sequential(
        (0): Conv2d(64, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (1): BatchNorm2d(64, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
        (2): ReLU(inplace=True)
        (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (4): BatchNorm2d(64, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
        (5): ReLU(inplace=True)
        (6): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (7): BatchNorm2d(64, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
        (8): ReLU(inplace=True)
   

In [3]:
import numpy as np


arr = np.array(np.random.rand(8,3))
offset = np.array([0,10,0])

In [3]:
def f(x):
    print(x)

eval('f'(5))

<>:4: SyntaxWarning: 'str' object is not callable; perhaps you missed a comma?
<>:4: SyntaxWarning: 'str' object is not callable; perhaps you missed a comma?
/tmp/ipykernel_3561194/2971988303.py:4: SyntaxWarning: 'str' object is not callable; perhaps you missed a comma?
  eval('f'(5))
/tmp/ipykernel_3561194/2971988303.py:4: SyntaxWarning: 'str' object is not callable; perhaps you missed a comma?
  eval('f'(5))
/tmp/ipykernel_3561194/2971988303.py:4: SyntaxWarning: 'str' object is not callable; perhaps you missed a comma?
  eval('f'(5))
/tmp/ipykernel_3561194/2971988303.py:4: SyntaxWarning: 'str' object is not callable; perhaps you missed a comma?
  eval('f'(5))


TypeError: 'str' object is not callable

In [ ]:
from nuscenes.nuscenes import NuScenes
nusc = NuScenes(version='v1.0-mini', dataroot='path/to/nuscenes/data', verbose=True)


: 

In [3]:
#@title Initial setup
from typing import Optional
import warnings
# Disable annoying warnings from PyArrow using under the hood.
warnings.simplefilter(action='ignore', category=FutureWarning)


import tensorflow as tf
import dask.dataframe as dd
from waymo_open_dataset import v2
from glob import glob

# Path to the directory with all components
dataset_dir = '/mnt/ssd2/waymo/training'

context_name = '10023947602400723454_1120_000_1140_000'

def read(tag: str) -> dd.DataFrame:
  """Creates a Dask DataFrame for the component specified by its tag."""
  print('Reading',f'{dataset_dir}/{tag}/{context_name}.parquet')
  paths = tf.io.gfile.glob(f'{dataset_dir}/{tag}/*.parquet')#{context_name}
  print(paths)
  return dd.read_parquet(paths)

2023-10-31 13:08:24.608536: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-10-31 13:08:24.814360: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-10-31 13:08:25.935087: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-10-31 13:08:25.935170: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] 

In [25]:
lidar_df = read('lidar')
lidar_pose_df = read('lidar_pose')
lidar_calibration_df = read('lidar_calibration')
lidar_segmentation_df = read('lidar_segmentation')
stats_df = read('stats')
vehicle_pose_df = read('vehicle_pose')


_, lidar_row = next(iter(lidar_df.iterrows()))
_, lidar_pose_row = next(iter(lidar_pose_df.iterrows()))
_, lidar_calibration_row = next(iter(lidar_calibration_df.iterrows()))
_, lidar_segmentation_row = next(iter(lidar_segmentation_df.iterrows()))
_, vehicle_pose_row = next(iter(vehicle_pose_df.iterrows()))

lidar = v2.LiDARComponent.from_dict(lidar_row)
lidar_pose = v2.LiDARPoseComponent.from_dict(lidar_pose_row)
lidar_calibration = v2.LiDARCalibrationComponent.from_dict(lidar_calibration_row)
lidar_segmentation = v2.LiDARSegmentationLabelComponent.from_dict(lidar_segmentation_row)
vehicle_pose = v2.VehiclePoseComponent.from_dict(vehicle_pose_row)

Reading /mnt/ssd2/waymo/training/lidar/10023947602400723454_1120_000_1140_000.parquet
['/mnt/ssd2/waymo/training/lidar/17356174167372765800_1720_000_1740_000.parquet', '/mnt/ssd2/waymo/training/lidar/16262849101474060261_3459_585_3479_585.parquet', '/mnt/ssd2/waymo/training/lidar/14742731916935095621_1325_000_1345_000.parquet', '/mnt/ssd2/waymo/training/lidar/1972128316147758939_2500_000_2520_000.parquet', '/mnt/ssd2/waymo/training/lidar/13585809231635721258_1910_770_1930_770.parquet', '/mnt/ssd2/waymo/training/lidar/4672649953433758614_2700_000_2720_000.parquet', '/mnt/ssd2/waymo/training/lidar/16102220208346880_1420_000_1440_000.parquet', '/mnt/ssd2/waymo/training/lidar/3132521568089292927_2220_000_2240_000.parquet', '/mnt/ssd2/waymo/training/lidar/15241656472211725662_2500_000_2520_000.parquet', '/mnt/ssd2/waymo/training/lidar/14183710428479823719_3140_000_3160_000.parquet', '/mnt/ssd2/waymo/training/lidar/2899357195020129288_3723_163_3743_163.parquet', '/mnt/ssd2/waymo/training/lid

In [26]:
from waymo_open_dataset.v2.perception.utils import lidar_utils
points = lidar_utils.convert_range_image_to_point_cloud(lidar.range_image_return1,
                                                        lidar_calibration,
                                                        lidar_pose.range_image_return1,
                                                        vehicle_pose,
                                                        False)


2023-10-31 13:20:29.743286: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudnn.so.8'; dlerror: libcudnn.so.8: cannot open shared object file: No such file or directory
2023-10-31 13:20:29.743316: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1934] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...
2023-10-31 13:20:29.744810: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [27]:
points.shape

TensorShape([153271, 3])

In [7]:
df = v2.merge(lidar_df, lidar_box_df, left_group=True, right_group=True)

In [5]:
df2 = v2.merge(lidar_df, lidar_box_df)

In [6]:
df2.head()

: 

In [6]:
lidar_df.head()

,key.segment_context_name,key.frame_timestamp_micros,key.laser_name,[LiDARComponent].range_image_return1.values,[LiDARComponent].range_image_return1.shape,[LiDARComponent].range_image_return2.values,[LiDARComponent].range_image_return2.shape
index,,,,,,,
15832924468527961_1564_160_1584_160;1507678826876435,15832924468527961_1564_160_1584_160,1507678826876435,1,"[-1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1....","[64, 2650, 4]","[-1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1....","[64, 2650, 4]"
15832924468527961_1564_160_1584_160;1507678826876435,15832924468527961_1564_160_1584_160,1507678826876435,2,"[-1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1....","[200, 600, 4]","[-1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1....","[200, 600, 4]"
15832924468527961_1564_160_1584_160;1507678826876435,15832924468527961_1564_160_1584_160,1507678826876435,3,"[-1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1....","[200, 600, 4]","[-1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1....","[200, 600, 4]"
15832924468527961_1564_160_1584_160;1507678826876435,15832924468527961_1564_160_1584_160,1507678826876435,4,"[-1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1....","[200, 600, 4]","[-1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1....","[200, 600, 4]"
15832924468527961_1564_160_1584_160;1507678826876435,15832924468527961_1564_160_1584_160,1507678826876435,5,"[-1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1....","[200, 600, 4]","[-1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1....","[200, 600, 4]"


In [4]:
_, row = next(iter(df2.iterrows()))
lidar = v2.LiDARComponent.from_dict(row)

KeyboardInterrupt: 